<a href="https://colab.research.google.com/github/fjadidi2001/Denoise/blob/main/Enhanced_Lightweight_Image_Denoising_Networks_with_Adaptive_Feature_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import random
import numpy as np
import torch

# Set base directory
base_dir = '/content/drive/MyDrive/SSNet'
os.makedirs(f'{base_dir}/data', exist_ok=True)
os.makedirs(f'{base_dir}/output', exist_ok=True)
os.makedirs(f'{base_dir}/saved_models', exist_ok=True)

# Set random seeds for reproducibility
def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed_torch()

print("Project Setup Complete!")
print(f"Base directory: {base_dir}")
print(f"CUDA available: {torch.cuda.is_available()}")

Project Setup Complete!
Base directory: /content/drive/MyDrive/SSNet
CUDA available: True
